## Why Analog AI

In-memory computing hardware increases the speed and energy-efficiency needed for the next steps in AI. Analog AI delivers radical performance improvements by combining compute and memory in a single device, eliminating the von Neumann bottleneck. 
Based on von Neumann architecture, conventional computers perform calculations by repeatedly transferring data between the memory and processor. These trips require time and energy, negatively impacting performance. This is known as the von Neumann bottleneck. 

<img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/processing-unit-and-computional-memory.png?raw=1" style="width:30%; height:30%" caption="Processing unit and conventional memory"/> 
<img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/processing-unit-and-conventional-memory.png?raw=1" style="width:30%; height:30%"/> 

By leveraging the physical properties of in-memory computing devices (example: Phase Change Memory or PCM), computation happens at the same place where the data is stored, drastically reducing energy consumption. Because there is no movement of data, tasks can be performed in a fraction of the time and with much less energy. This is different from a conventional computer, where the data is transferred from the DRAM memory to the CPU every time a computation is done. For example, moving 64 bits of data from DRAM to CPU consumes 1-2nJ, which is 10,000–2,000,000 times more energy than is dissipated in a PCM device performing a multiplication operation (1-100fJ). Also, PCM does not consume power when the devices are inactive, and the data will be retained for up to 10 years even when the power supply is turned off. 

## The physics behind PCM

 With PCM, when an electrical pulse is applied to the material, it changes the conductance of the device by switching the material between amorphous and crystalline phases. A low electrical pulse will make the PCM device more crystalline (less resistance), this pulse can be repeatedly applied to gradually decrease the device resistance. On the other hand, the material change from the crystalline phase (low resistance) to the amorphouse phase (high resistance) is quite abrupt and requires a high electrical pulse to RESET the device. Therefore, it is possible to records the states as a continuum of values between the two extremes, instead of encoding 0 or 1 like in the digital world.

PCM devices have the ability to store synaptic weights in their analog conductance state. When PCM devices are arranged in a crossbar configuration, it allows to perform an analog matrix-vector multiplication in a single time step, exploiting the advantages of multi-level storage capability and Kirchhoff’s circuits laws. The figure below shows how PCM devices are arranged in a crossbar configuration. 
<center><img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/pcm-array.png?raw=1" style="width:30%; height:30%"/></center> 

This crossbar configuration is also referred to as an Analog tile. The PCM devices at each crossbard crosspoint are also referred to as Resitive Processing Units or RPU units as shown in the figure below:

<center><img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/pcm_rpu_unit.png?raw=1" style="width:30%; height:30%"/></center> 

Besides PCM, other devices or materials can be used as resistive units or RPUs in the crossbar configuration. Examples include Resistive Random Access Memory (RRAM), Electro chemical Random Access Memory (ECRAM), Magnetic RAM (MRAM), photonics, etc. 


## Analog AI and Neural Networks

In deep learning inference, data propagation through multiple layers of a neural network involves a sequence of matrix multiplications, as each layer can be represented as a matrix of synaptic weights. On an Analog chip, these weights are stored in the conductance states of resistive devices such as PCM. The devices are arranged in crossbar arrays, creating an artificial neural network where all matrix multiplications are performed in-place in an analog manner. This structure allows inference to be performed using little energy with high areal density of synapses. An in-memory computing chip typically consists of multiple crossbar arrays of memory devices that communicate with each other (see figure below). A neural network layer can be implemented on (at least) one crossbar, in which the weights of that layer are stored in the charge or conductance state of the memory devices at the crosspoints.

<center><img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/analog_Dnn.png?raw=1" style="width:60%; height:60%"/></center> 



## IBM Analog Hardware Acceleration Kit (AIHWKIT)

The IBM Analog Hardware Acceleration Kit (AIHWKIT) is an open source Python toolkit for exploring and using the capabilities of in-memory computing devices such as PCM in the context of artificial intelligence. 
The pytorch integration consists of a series of primitives and features that allow using the toolkit within PyTorch. 

The github repository can be found at: https://github.com/IBM/aihwkit

To learn more about Analog AI and the harware befind it, refer to this webpage: https://analog-ai-demo.mybluemix.net/hardware



The first thing to do is to install the AIHHKIT and dependencies in your environment. The preferred way to install this package is by using the Python package index:

In [1]:
pip install aihwkit

Note: you may need to restart the kernel to use updated packages.


In this notebook we will use the AIHWKIT to train a LeNet5 inspired analog network, as studied in the paper: https://www.frontiersin.org/articles/10.3389/fnins.2017.00538/full

<img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/LeNet5_animation.png?raw=1" style="width:40%; height:40%"/> 

The network will be trained using the MNIST dataset, a collection of images representing the digits 0 to 9. The architecture of the LeNet5 network is shown below:

<img src="https://github.com/kaoutar55/aihwkit/blob/master/notebooks/LeNet.png?raw=1" style="width:60%; height:60%"/> 

## Analog layers
If the library was installed correctly, you can use the following snippet for creating an analog layer and predicting the output:

In [2]:
from torch import Tensor
from aihwkit.nn import AnalogLinear

model = AnalogLinear(2, 2)
model(Tensor([[0.1, 0.2], [0.3, 0.4]]))

tensor([[0.4235, 0.0471],
        [0.2824, 0.0941]], grad_fn=<AnalogFunctionBackward>)

## RPU Configuration
Now that the package is installed and running, we can start working on creating the LeNet5 network.

AIHWKIT offers different Analog layers that can be used to build a network, including AnalogLinear and AnalogConv2d which will be the main layers used to build the present network. 
In addition to the standard input that are expected by the PyTorch layers (in_channels, out_channels, etc.) the analog layers also expect a rpu_config input which defines various settings of the RPU tile. 

Through the rpu_config parameter the user can specify many of the hardware specs such as: device used in the cross-point array, bit used by the ADC/DAC converters, noise values and many other. 

Additional details on the RPU configuration can be found at https://aihwkit.readthedocs.io/en/latest/using_simulator.html#rpu-configurations

For this particular case we will define a RPU which uses RRAM devices in the cross-point array:

In [3]:
def create_rpu_config():

    from aihwkit.simulator.presets import ReRamSBPreset

    rpu_config=ReRamSBPreset()

    return rpu_config

We can now use this rpu_config as input of the network model:

In [4]:
from torch.nn import Tanh, MaxPool2d, LogSoftmax, Flatten
from aihwkit.nn import AnalogConv2d, AnalogLinear, AnalogSequential

def create_analog_network(rpu_config):
    
    channel = [16, 32, 512, 128]
    model = AnalogSequential(
        AnalogConv2d(in_channels=1, out_channels=channel[0], kernel_size=5, stride=1,
                        rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        AnalogConv2d(in_channels=channel[0], out_channels=channel[1], kernel_size=5, stride=1,
                        rpu_config=rpu_config),
        Tanh(),
        MaxPool2d(kernel_size=2),
        Tanh(),
        Flatten(),
        AnalogLinear(in_features=channel[2], out_features=channel[3], rpu_config=rpu_config),
        Tanh(),
        AnalogLinear(in_features=channel[3], out_features=10, rpu_config=rpu_config),
        LogSoftmax(dim=1)
    )

    return model

## Analog Optimizer 

We will use the cross entropy to calculate the loss and the Stochastic Gradient Descent (SGD) as optimizer:

In [5]:
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()


from aihwkit.optim import AnalogSGD

def create_analog_optimizer(model):
    """Create the analog-aware optimizer.

    Args:
        model (nn.Module): model to be trained

    Returns:
        Optimizer: created analog optimizer
    """
    
    optimizer = AnalogSGD(model.parameters(), lr=0.01) # we will use a learning rate of 0.01 as in the paper
    optimizer.regroup_param_groups(model)

    return optimizer

## Training the network

We can now write the train function which will optimize the network over the MNIST train dataset. The train_step function will take as input the images to train on, the model to train and the criterion and optimizer to train with:

In [6]:
from torch import device, cuda

DEVICE = device('cuda' if cuda.is_available() else 'cpu')
print('Running the simulation on: ', DEVICE)

def train_step(train_data, model, criterion, optimizer):
    """Train network.

    Args:
        train_data (DataLoader): Validation set to perform the evaluation
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss
        optimizer (Optimizer): analog model optimizer

    Returns:
        train_dataset_loss: epoch loss of the train dataset
    """
    total_loss = 0

    model.train()

    for images, labels in train_data:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        # Add training Tensor to the model (input).
        output = model(images)
        loss = criterion(output, labels)

        # Run training (backward propagation).
        loss.backward()

        # Optimize weights.
        optimizer.step()
        total_loss += loss.item() * images.size(0)
    train_dataset_loss = total_loss / len(train_data.dataset)

    return train_dataset_loss

Running the simulation on:  cpu


Since training can be quite time consuming it is nice to see the evolution of the training process by testing the model capabilities on a set of images that it has not seen before (test dataset). So we write a test_step function:

In [7]:
def test_step(validation_data, model, criterion):
    """Test trained network

    Args:
        validation_data (DataLoader): Validation set to perform the evaluation
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss

    Returns: 
        test_dataset_loss: epoch loss of the train_dataset
        test_dataset_error: error of the test dataset
        test_dataset_accuracy: accuracy of the test dataset
    """
    total_loss = 0
    predicted_ok = 0
    total_images = 0

    model.eval()

    for images, labels in validation_data:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        pred = model(images)
        loss = criterion(pred, labels)
        total_loss += loss.item() * images.size(0)

        _, predicted = torch.max(pred.data, 1)
        total_images += labels.size(0)
        predicted_ok += (predicted == labels).sum().item()
        test_dataset_accuracy = predicted_ok/total_images*100
        test_dataset_error = (1-predicted_ok/total_images)*100

    test_dataset_loss = total_loss / len(validation_data.dataset)

    return test_dataset_loss, test_dataset_error, test_dataset_accuracy

To reach satisfactory accuracy levels, the train_step will have to be repeated mulitple time so we will implement a loop over a certain number of epochs:

In [8]:
def training_loop(model, criterion, optimizer, train_data, validation_data, epochs=10, print_every=1):
    """Training loop.

    Args:
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss
        optimizer (Optimizer): analog model optimizer
        train_data (DataLoader): Validation set to perform the evaluation
        validation_data (DataLoader): Validation set to perform the evaluation
        epochs (int): global parameter to define epochs number
        print_every (int): defines how many times to print training progress

    Returns:
        nn.Module, Optimizer, Tuple: model, optimizer, and a tuple of
            lists of train losses, validation losses, and test error
    """
    train_losses = []
    valid_losses = []
    test_error = []

    # Train model
    for epoch in range(0, epochs):
        # Train_step
        train_loss = train_step(train_data, model, criterion, optimizer)
        train_losses.append(train_loss)

        if epoch % print_every == (print_every - 1):
            # Validate_step
            with torch.no_grad():
                valid_loss, error, accuracy = test_step(validation_data, model, criterion)
                valid_losses.append(valid_loss)
                test_error.append(error)

            print(f'Epoch: {epoch} --- '
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Test error: {error:.2f}%\t'
                  f'Test accuracy: {accuracy:.2f}%\t')

We will now download the MNIST dataset and prepare the images for the training and test:

In [9]:
import os
from torchvision import datasets, transforms
PATH_DATASET = os.path.join('data', 'DATASET')
os.makedirs(PATH_DATASET, exist_ok=True)

def load_images():
    """Load images for train from torchvision datasets."""

    transform = transforms.Compose([transforms.ToTensor()])
    train_set = datasets.MNIST(PATH_DATASET, download=True, train=True, transform=transform)
    test_set = datasets.MNIST(PATH_DATASET, download=True, train=False, transform=transform)
    train_data = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)
    test_data = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)

    return train_data, test_data

Put together all the code above to train

In [10]:
import torch

torch.manual_seed(1)

#load the dataset
train_data, test_data = load_images()

#create the rpu_config
rpu_config = create_rpu_config()

#create the model
model = create_analog_network(rpu_config).to(DEVICE)

#define the analog optimizer
optimizer = create_analog_optimizer(model)

training_loop(model, criterion, optimizer, train_data, test_data)

100.0%


Extracting data/DATASET/MNIST/raw/train-images-idx3-ubyte.gz to data/DATASET/MNIST/raw


102.8%


Extracting data/DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to data/DATASET/MNIST/raw


11.7%

100.0%


Extracting data/DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to data/DATASET/MNIST/raw



112.7%
/opt/miniconda3/envs/vs/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/DATASET/MNIST/raw

Processing...
Done!
